In [ ]:
FILE = "../data/osm/NewYork.osm"
with open(FILE, encoding="utf-8") as f:
    data_ny = f.readlines()

In [ ]:
import os, math

In [ ]:
import os, math
size = os.path.getsize(FILE) / math.pow(1024, 3)
print("{:.3} GB".format(size))

In [ ]:
[x.strip() for x in data_ny[239:246]]

In [ ]:
data_ny[240]

In [ ]:
data_ny[240].split('"')

In [ ]:
from operator import itemgetter
extract_tag = itemgetter(1, 3)

In [ ]:
extract_tag([1, 2, 3, 4])

In [ ]:
tags_ny = [extract_tag(line.split('"')) \
           for line in data_ny \
           if "tag" in line ]

In [ ]:
tags_ny[1:100000:10000]

In [ ]:
from collections import Counter
c = Counter(tags_ny)

In [ ]:
c.most_common(5)

In [ ]:
DIR = "../data/osm/southamerica"
size = sum(
    os.path.getsize(os.path.join(DIR, f)) \
    for f in os.listdir(DIR)
) / math.pow(1024, 3)
print("{:.3} GB".format(size))

In [ ]:
lines = []
for file in os.listdir(DIR):
    with open(os.path.join(DIR, file), encoding="utf-8") as f:
        lines.append(f.readlines())

In [ ]:
import dask.bag as db

lst = [1, 2, 3, 4]
bag = db.from_sequence(lst, npartitions=1)
bag

In [ ]:
bag.map(lambda x: x + 1)

In [ ]:
bag.map(lambda x: x + 1).compute()

In [ ]:
bag.fold(lambda acc, e: acc + e, initial=0).compute()

In [ ]:
bag.sum().compute()

In [ ]:
bag.fold(lambda acc, e: acc * e, initial=0).compute()

In [ ]:
bag.foldby(lambda x: x % 2 == 0,
           lambda x, y: x + y,
           initial=0).compute()

In [ ]:
from dask.distributed import Client
client = Client()

In [ ]:
data = db.read_text("../data/osm/southamerica/southam*")
data

In [ ]:
task = data.filter(
            lambda x: "tag" in x
        ).foldby(
            lambda x: extract_tag(x.split('"')),
            lambda x, _: x + 1, 0
        ).topk(10, itemgetter(1))
task

In [ ]:
from dask.diagnostics import ProgressBar
with ProgressBar():
    counts = task.compute()

In [ ]:
list(counts)